Option chains
=======

In [11]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15, readonly=True )

<IB connected to 127.0.0.1:7497 clientId=15>

Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [12]:
spx = Stock('SPY', 'SMART', currency='USD')

ib.qualifyContracts(spx)

[Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY')]

To avoid issues with market data permissions, we'll use delayed data:

In [13]:
ib.reqMarketDataType(1)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [14]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Stock(conId=756733, symbol='SPY', exchange='SMART', primaryExchange='ARCA', currency='USD', localSymbol='SPY', tradingClass='SPY'), time=datetime.datetime(2020, 5, 7, 21, 20, 27, 976624, tzinfo=datetime.timezone.utc), bid=288.41, bidSize=1, ask=288.46, askSize=1, last=288.41, lastSize=1, volume=747137, open=287.79, high=289.78, low=287.13, close=284.25, halted=0.0)

Take the current market value of the ticker:

In [15]:
spxValue = ticker.marketPrice()
spxValue

288.41

The following request fetches a list of option chains:

In [16]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,NASDAQOM,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
1,PSE,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
2,BATS,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
3,AMEX,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
4,PEARL,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
5,NASDAQBX,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
6,ISE,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
7,MIAX,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
8,SMART,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."
9,CBOE,756733,SPY,100,"[20200508, 20200511, 20200513, 20200515, 20200...","[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0..."


In this case we're only interested in the montly options trading on SMART:

In [17]:
chains = [ c for c in chains if c.tradingClass == 'SPY' and c.exchange == 'SMART']

#print(len(chains))


chain = chains[0]

chain


OptionChain(exchange='SMART', underlyingConId='756733', tradingClass='SPY', multiplier='100', expirations=['20200508', '20200511', '20200513', '20200515', '20200518', '20200520', '20200522', '20200526', '20200527', '20200529', '20200601', '20200603', '20200605', '20200608', '20200610', '20200612', '20200619', '20200626', '20200630', '20200717', '20200821', '20200918', '20200930', '20201016', '20201120', '20201218', '20201231', '20210115', '20210319', '20210331', '20210618', '20210917', '20211217', '20220121', '20220318', '20221216'], strikes=[5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 105.0, 110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 159.0, 160.0, 161.0, 162.0, 163.0, 164.0, 165.0, 166.0, 167.0, 168.0, 169.0, 170.0, 171.0, 172.0, 173.0, 174.0, 175.0, 176.0, 177.0, 178.0, 179.0, 180.0, 181.0, 182.0, 183.0, 18

What we have here is the full matrix of expirations x strikes. 

From this we can build all PUT  option contracts that meet our conditions (PUTS):

In [18]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P']

putContracts = [Option('SPY', expiration, strike, right, 'SMART', tradingClass='SPY')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*putContracts)
len(putContracts)

24

Select one PUT option contract as 1st leg

In [19]:

legOne = putContracts[18]

[ legOneTicker ] = ib.reqTickers( legOne )

legOne

Option(conId=416199316, symbol='SPY', lastTradeDateOrContractMonth='20200513', strike=280.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   200513P00280000', tradingClass='SPY')

In [29]:
#legOneTicker.marketPrice()

legOneTicker.last

1.42

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [43]:
print(legOneTicker.hasBidAsk())

print(legOneTicker.lastGreeks)

False
None


build first combo leg

In [21]:
comboLegOne = ComboLeg(conId=legOne.conId, ratio= 1, action='SELL', exchange='SMART' )


Select second put contract as second leg

In [22]:
legTwo = putContracts[16]

[ legTwoTicker ] = ib.reqTickers( legTwo )

legTwo

Option(conId=416199295, symbol='SPY', lastTradeDateOrContractMonth='20200513', strike=270.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPY   200513P00270000', tradingClass='SPY')

In [30]:
legTwoTicker.last

0.41

In [41]:
print( legTwoTicker.hasBidAsk() )

print(legTwoTicker.lastGreeks)

False
None


Build second combo leg

In [25]:
comboLegTwo = ComboLeg(conId=legTwo.conId, ratio=1, action='BUY', exchange='SMART')

Build Contract

In [26]:
combo = Contract( symbol=spx.symbol, secType='BAG', currency='USD', exchange='SMART', comboLegs=[comboLegOne, comboLegTwo] )


#not valid since I think 'BAG' type of contracts don't really have a contract ID
#ib.qualifyContracts( combo )


Request combo ticker... So far unable to get last price or any info

In [27]:
[comboTicker] = ib.reqTickers( combo )

comboTicker.last

nan

Create / Prepare (save?) combo order

In [ ]:
#! [market]

MarketOrder( action="BUY", )

order = Order()
order.action = "SELL"
order.orderType = "MKT"
order.totalQuantity = 1

ib.
        

In [11]:
ib.disconnect()